In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm

BASE = "/media/giani/Gianicosas/Magister/Proyecto/raw_data"

# Proceso de matricula

## Datos

Se comparará la cantidad de alumnos por carrera por año para validación de datos

In [2]:
# Fuente: https://www.mifuturo.cl/bases-de-datos-de-matriculados/
proceso = pd.read_csv(f"{BASE}/OFICIAL_WEB_PROCESO_MAT_2007_AL_2022_24_06_2022_E.csv", encoding="latin-1", sep=";")
proceso.head()

/tmp/ipykernel_3966/618280487.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  proceso = pd.read_csv(f"{BASE}/OFICIAL_WEB_PROCESO_MAT_2007_AL_2022_24_06_2022_E.csv", encoding="latin-1", sep=";")


,AÑO,TOTAL MATRICULADOS,MATRICULADOS MUJERES POR PROGRAMA,MATRICULADOS HOMBRES POR PROGRAMA,TOTAL MATRICULADOS PRIMER AÑO,MATRICULADOS MUJERES PRIMER AÑO,MATRICULADOS HOMBRES PRIMER AÑO,CLASIFICACIÓN INSTITUCIÓN NIVEL 1,CLASIFICACIÓN INSTITUCIÓN NIVEL 2,CLASIFICACIÓN INSTITUCIÓN NIVEL 3,...,TES PARTICULAR SUBVENCIONADO,TES PARTICULAR PAGADO,TES CORP. DE ADMINISTRACIÓN DELEGADA,TES SERVICIO LOCAL EDUCACION,TOTAL TES,% COBERTURA TES,TIPO ESTABLECIMIENTO HC,TIPO ESTABLECIMIENTO TP,CLAS_EST ADULTO,CLAS_EST JOVEN
0,MAT_2022,2,2.0,NaN,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,2.0,NaN,NaN,NaN,2,"100,0%",2.0,NaN,NaN,2.0
1,MAT_2022,10,8.0,2.0,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,3.0,2.0,NaN,NaN,9,"90,0%",4.0,5.0,2.0,7.0
2,MAT_2022,34,NaN,34.0,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,14.0,3.0,NaN,NaN,26,"76,5%",17.0,9.0,4.0,22.0
3,MAT_2022,10,NaN,10.0,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,4.0,NaN,NaN,NaN,10,"100,0%",6.0,4.0,1.0,9.0
4,MAT_2022,76,4.0,72.0,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,29.0,6.0,4.0,NaN,71,"93,4%",30.0,41.0,4.0,67.0


In [41]:
proceso = proceso[proceso["CLASIFICACIÓN INSTITUCIÓN NIVEL 1"].isin(['Institutos Profesionales', 'Centros de Formación Técnica'])]

In [3]:
print(list(proceso.columns)), proceso.shape

['AÑO', 'TOTAL MATRICULADOS', 'MATRICULADOS MUJERES POR PROGRAMA', 'MATRICULADOS HOMBRES POR PROGRAMA', 'TOTAL MATRICULADOS PRIMER AÑO', 'MATRICULADOS MUJERES PRIMER AÑO', 'MATRICULADOS HOMBRES PRIMER AÑO', 'CLASIFICACIÓN INSTITUCIÓN NIVEL 1', 'CLASIFICACIÓN INSTITUCIÓN NIVEL 2', 'CLASIFICACIÓN INSTITUCIÓN NIVEL 3', 'CÓDIGO DE INSTITUCIÓN', 'NOMBRE INSTITUCIÓN', 'ACREDITACIÓN INSTITUCIONAL', 'REGIÓN', 'PROVINCIA', 'COMUNA', 'NOMBRE SEDE', 'NOMBRE CARRERA', 'ÁREA DEL CONOCIMIENTO', 'CINE-F 1997 ÁREA', 'CINE-F 1997 SUBAREA', 'ÁREA CARRERA GENÉRICA', 'CINE-F 2013 ÁREA', 'CINE-F 2013 SUBAREA', 'NIVEL GLOBAL', 'CARRERA CLASIFICACIÓN NIVEL 1', 'CARRERA CLASIFICACIÓN NIVEL 2', 'MODALIDAD', 'JORNADA', 'TIPO DE PLAN DE LA CARRERA', 'DURACIÓN ESTUDIO CARRERA', 'DURACIÓN TOTAL DE CARRERA', 'CÓDIGO CARRERA', 'ACREDITACIÓN CARRERA', 'TOTAL RANGO DE EDAD', 'RANGO DE EDAD 15 A 19 AÑOS', 'RANGO DE EDAD 20 A 24 AÑOS', 'RANGO DE EDAD 25 A 29 AÑOS', 'RANGO DE EDAD 30 A 34 AÑOS', 'RANGO DE EDAD 35 A 39 AÑ

(None, (216153, 56))

In [4]:
proceso["AÑO"].value_counts().sort_index(ascending=False)

MAT_2022    15631
MAT_2021    15401
MAT_2020    14381
MAT_2019    14546
MAT_2018    14912
MAT_2017    14730
MAT_2016    14750
MAT_2015    14788
MAT_2014    14645
MAT_2013    14285
MAT_2012    13270
MAT_2011    12514
MAT_2010    12147
MAT_2009    11414
MAT_2008     9943
MAT_2007     8796
Name: AÑO, dtype: int64

# Matriculados

### Todas las columnas
```python
# pd.read_csv(f"{BASE}/20220719_Matrícula_Ed_Superior_2022_PUBL_MRUN.csv", nrows=5, sep=";").columns
[
    'cat_periodo', 'id', 'codigo_unico', 'mrun', 'gen_alu', 'fec_nac_alu', 'rango_edad', 'anio_ing_carr_ori',
    'sem_ing_carr_ori', 'anio_ing_carr_act', 'sem_ing_carr_act', 'anio_ing_carr_act', 'tipo_inst_1', 'tipo_inst_2', 'tipo_inst_3',
    'cod_inst', 'nomb_inst', 'cod_sede', 'nomb_sede', 'cod_carrera', 'nomb_carrera', 'modalidad', 'jornada',
    'version', 'tipo_plan_carr', 'dur_estudio_carr', 'dur_proceso_tit', 'dur_total_carr', 'region_sede',
    'provincia_sede', 'comuna_sede', 'nivel_global', 'nivel_carrera_1', 'nivel_carrera_2', 'requisito_ingreso',
    'vigencia_carrera', 'formato_valores', 'valor_matricula', 'valor_arancel', 'codigo_demre', 'area_conocimiento',
    'cine_f_97_area', 'cine_f_97_subarea', 'area_carrera_generica', 'cine_f_13_area', 'cine_f_13_subarea',
    'acreditada_carr', 'acreditada_inst', 'acre_inst_desde_hasta', 'acre_inst_anio', 'costo_proceso_titulacion',
    'costo_obtencion_titulo_diploma', 'forma_ingreso'
]
```

### Cols para determinar vector objetivo
```python
['cat_periodo', 'codigo_unico', 'mrun', 'anio_ing_carr_ori', 'anio_ing_carr_act', 'tipo_inst_1', 'dur_estudio_carr']
```

### Tipos de instituciones
```python
# Salida: ['Universidades', 'Institutos Profesionales', 'Centros de Formación Técnica']
pd.concat([
    pd.read_csv(f, sep=";", usecols=["tipo_inst_1"])
    for f in tqdm(glob(f"{BASE}/20220719_Matrícula_Ed_Superior_*.csv"), total=16)
])["tipo_inst_1"].unique()
instituciones = ['Institutos Profesionales', 'Centros de Formación Técnica']
```

## Lectura de datos matriculados

Fuente: https://datosabiertos.mineduc.cl/matricula-en-educacion-superior/

In [5]:
def iter_df_read(file, col_name, condition_list, chunksize=1000, **kwargs):
    iter_csv = pd.read_csv(file, iterator=True, chunksize=chunksize, **kwargs)
    return pd.concat([chunk[chunk[col_name].isin(condition_list)] for chunk in iter_csv])

In [6]:
cols_time = ['cat_periodo', 'codigo_unico', 'mrun', 'anio_ing_carr_ori', 'anio_ing_carr_act', 'tipo_inst_1', 'dur_estudio_carr']
instituciones = ['Institutos Profesionales', 'Centros de Formación Técnica']

matriculados = pd.concat([
    iter_df_read(f, "tipo_inst_1", instituciones, sep=";", usecols=cols_time)
    for f in tqdm(glob(f"{BASE}/post_2015/20220719_Matrícula_Ed_Superior_*.csv"), total=8)
])
matriculados.head()

  0%|          | 0/8 [00:00<?, ?it/s]

,cat_periodo,codigo_unico,mrun,anio_ing_carr_ori,anio_ing_carr_act,tipo_inst_1,dur_estudio_carr
0,2015,I374S1C9J1V1,5.0,2015,NaN,Centros de Formación Técnica,5
1,2015,I498S6C132J2V1,37.0,2015,NaN,Centros de Formación Técnica,5
5,2015,I111S12C104J1V1,107.0,2013,NaN,Institutos Profesionales,9
7,2015,I111S14C309J2V1,118.0,2015,NaN,Institutos Profesionales,5
9,2015,I260S44C120J2V1,184.0,2014,NaN,Centros de Formación Técnica,5


### Información del DataFrame

In [7]:
matriculados.shape

(4133070, 7)

In [8]:
matriculados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4133070 entries, 0 to 1301920
Data columns (total 7 columns):
 #   Column             Dtype  
---  ------             -----  
 0   cat_periodo        int64  
 1   codigo_unico       object 
 2   mrun               float64
 3   anio_ing_carr_ori  int64  
 4   anio_ing_carr_act  float64
 5   tipo_inst_1        object 
 6   dur_estudio_carr   int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 252.3+ MB


### Exploración de nulos en año de ingreso a carrera actual

In [9]:
matriculados.isna().sum()

cat_periodo                0
codigo_unico               0
mrun                    1461
anio_ing_carr_ori          0
anio_ing_carr_act    1193783
tipo_inst_1                0
dur_estudio_carr           0
dtype: int64

In [38]:
for a in range(2015, 2023):
    t = matriculados[matriculados['cat_periodo']==a]['anio_ing_carr_act']
    print(f"Cantidad de datos Año {a}: {t.shape[0]}. Cantidad nulos: {t.isna().sum()}. Porcentaje nulos: {round(t.isna().sum() / t.shape[0] * 100, 2)}%")

Cantidad de datos Año 2015: 525232. Cantidad nulos: 347990. Porcentaje nulos: 66.25%
Cantidad de datos Año 2016: 526241. Cantidad nulos: 431996. Porcentaje nulos: 82.09%
Cantidad de datos Año 2017: 513947. Cantidad nulos: 413447. Porcentaje nulos: 80.45%
Cantidad de datos Año 2018: 511481. Cantidad nulos: 0. Porcentaje nulos: 0.0%
Cantidad de datos Año 2019: 519131. Cantidad nulos: 0. Porcentaje nulos: 0.0%
Cantidad de datos Año 2020: 492182. Cantidad nulos: 0. Porcentaje nulos: 0.0%
Cantidad de datos Año 2021: 514163. Cantidad nulos: 0. Porcentaje nulos: 0.0%
Cantidad de datos Año 2022: 529232. Cantidad nulos: 0. Porcentaje nulos: 0.0%


### Pre procesamiento: Cursando carrera

Se crea columna `cursando`, la cual indica:
- `1` si los años transcurridos desde el año de ingreso a la carrera hasta el año 2022 es mayor a la duración de la carrera en años.
- `0` en caso contrario

Esto se utilizará más adelante en la construcción del vector objetivo; Se hace una selección de datos donde solo se selecciona a aquellos que tengan `cursando==0`, para medir inicialmente deserción solamente en aquellos que no se encuentren ya en la ventana de duración de la carrera.

In [11]:
matriculados = matriculados.dropna(subset=["mrun"]).reset_index(drop=True)
matriculados["cursando"] = np.where(
    pd.Series([2022] * matriculados.shape[0]) - matriculados["anio_ing_carr_ori"] > matriculados["dur_estudio_carr"] / 2, 0, 1
)

matriculados["cursando"].value_counts("%")

0    0.657826
1    0.342174
Name: cursando, dtype: float64

### Validación: Cantidad de matriculados por carrera por año

Se compara con la base de datos agregados

In [12]:
anios_proceso = sorted(proceso["AÑO"].unique(), reverse=True)[:8]
assert sorted(matriculados["cat_periodo"].unique(), reverse=True) == [int(a.split("_")[1]) for a in anios_proceso]

In [13]:
# Cantidad de matriculados por año y carrera
mat_por_carrera = matriculados.groupby(
    ["cat_periodo", "codigo_unico"]
).agg({"mrun": pd.Series.nunique}).rename(columns={"mrun": "cant_matriculados_carrera_anio"}).reset_index().sort_values(by=["cant_matriculados_carrera_anio"])
mat_por_carrera["cant_matriculados_carrera_anio"].describe()

count    54938.000000
mean        75.199971
std        103.506341
min          1.000000
25%         16.000000
50%         47.000000
75%         98.000000
max       2860.000000
Name: cant_matriculados_carrera_anio, dtype: float64

In [42]:
proceso[
    proceso["AÑO"].isin(anios_proceso)
].groupby([
    "AÑO", "CÓDIGO CARRERA"
]).agg({
    "TOTAL MATRICULADOS": "sum"
}).reset_index()["TOTAL MATRICULADOS"].describe()

count    54938.000000
mean        75.231534
std        103.523596
min          1.000000
25%         16.000000
50%         47.000000
75%         98.000000
max       2860.000000
Name: TOTAL MATRICULADOS, dtype: float64

- La distribución de matriculados por carrera por año coincide entre los datos agregados y los desagregados

In [43]:
mat_por_carrera["codigo_unico"].nunique(), proceso[(proceso["AÑO"].isin("MAT_" + mat_por_carrera["cat_periodo"].astype(str)))]["CÓDIGO CARRERA"].nunique()

(13055, 13055)

- Las carreras coinciden en ambas bases:

In [44]:
print(list(set(proceso[(proceso["AÑO"].isin("MAT_" + mat_por_carrera["cat_periodo"].astype(str)))]["CÓDIGO CARRERA"].unique())-set(mat_por_carrera["codigo_unico"].unique()))[:10])

[]


- Se explora la diferencia de matriculados

In [46]:
coinciden = proceso[
    (proceso["AÑO"].isin("MAT_" + mat_por_carrera["cat_periodo"].astype(str)))
    & (proceso["CÓDIGO CARRERA"].isin(mat_por_carrera["codigo_unico"]))
].reset_index(drop=True).sort_values("TOTAL MATRICULADOS")

cant_mat_dif = coinciden[
    coinciden["TOTAL MATRICULADOS"] != mat_por_carrera["cant_matriculados_carrera_anio"].reset_index(drop=True).values
][["AÑO", "CÓDIGO CARRERA", "TOTAL MATRICULADOS"]].rename(columns={"CÓDIGO CARRERA": "codigo_unico"})
cant_mat_dif["cat_periodo"] = cant_mat_dif["AÑO"].str.split("_", expand=True)[1].astype(int)

difieren = pd.merge(cant_mat_dif, mat_por_carrera)
difieren["diferencia"] = difieren["TOTAL MATRICULADOS"] - difieren["cant_matriculados_carrera_anio"]
difieren["diferencia"].describe()

count    1731.000000
mean        0.095898
std         0.457488
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         6.000000
Name: diferencia, dtype: float64

In [49]:
difieren.sort_values(by="diferencia", ascending=False).head()

,AÑO,codigo_unico,TOTAL MATRICULADOS,cat_periodo,cant_matriculados_carrera_anio,diferencia
710,MAT_2022,I155S2C58J1V1,91,2022,85,6
1655,MAT_2022,I155S2C87J2V1,330,2022,324,6
1531,MAT_2022,I155S2C7J1V1,254,2022,249,5
824,MAT_2020,I155S2C39J2V1,102,2020,97,5
1365,MAT_2018,I143S26C36J2V1,187,2018,183,4


- En general la cantidad de matriculados coincide, habiendo una diferencia máxima de 6 alumnos.

### Validación: Código único

- Se corrobora que un código único puede estar asociado a distintos nombres de carrera, pero se asumirá que todos los nombres diferentes hacen referencia a la misma.
- Todos los códigos únicos repetidos pertenecen a la misma institución
- Se observa un ejemplo

In [20]:
proceso.groupby(["CÓDIGO CARRERA"]).agg({"NOMBRE INSTITUCIÓN": pd.Series.nunique, "NOMBRE CARRERA": pd.Series.nunique}).reset_index().sort_values(by=["NOMBRE CARRERA"], ascending=False).head()

,CÓDIGO CARRERA,NOMBRE INSTITUCIÓN,NOMBRE CARRERA
35620,I71S1C221J2V1,1,6
34871,I70S1C2J1V1,1,6
35899,I71S1C70J2V1,1,6
6156,I11S1C77J2V1,1,6
30078,I45S4C287J1V1,1,5


In [21]:
proceso[proceso["CÓDIGO CARRERA"]=="I71S1C221J2V1"][["NOMBRE INSTITUCIÓN", "NOMBRE CARRERA", "CÓDIGO CARRERA"]].drop_duplicates()

,NOMBRE INSTITUCIÓN,NOMBRE CARRERA,CÓDIGO CARRERA
11677,UNIVERSIDAD DE SANTIAGO DE CHILE,MAGISTER EN EDUCACION MENCION EN GESTION Y LID...,I71S1C221J2V1
41896,UNIVERSIDAD DE SANTIAGO DE CHILE,MAGISTER EN EDUCACION,I71S1C221J2V1
71299,UNIVERSIDAD DE SANTIAGO DE CHILE,"MAGISTER EN EDUCACION, MENCION CURRICULUM Y EV...",I71S1C221J2V1
115921,UNIVERSIDAD DE SANTIAGO DE CHILE,"MAGISTER EN EDUCACION, MENCION CURRICULUM Y EV...",I71S1C221J2V1
170932,UNIVERSIDAD DE SANTIAGO DE CHILE,"MAGISTER EN EDUCACION, MENCION ORIENTACION EDU...",I71S1C221J2V1
182955,UNIVERSIDAD DE SANTIAGO DE CHILE,MAGISTER EN EDUCACION EN ORIENTACION EDUCACION...,I71S1C221J2V1


## Operacionalización de datos

### Alumno desertor
- Tiene que haber pasado un tiempo mayor a la duración de la carrera
- El último semestre cursado tiene que ser menor a la cantidad de semestres de la carrera (no se cuenta con este dato)
- Considerar todo para el mismo codigo de carrera cada vez

## Vector Objetivo: Primera iteración
- Se considera para esta primera iteración aquellos alumnos que ya hayan superado la ventana de tiempo de duración de la carrera.
- Como no se tiene el dato de qué número de semestre está cursando el alumno (considerando el total de semestres disponibles), se asumirá al menos en esta primera etapa que los alumnos no han repetido semestres/años. Esto claramente subestimará la cantidad de desertores, ya que un alumno puede haber completado la cantidad de años teóricos cursados, sin haber completado la carrera (por repetición de un ramo, congelamiento, etc), lo que lo calificaría como desertor.

In [22]:
# Algunos aparecen 2 veces probablemente porque se matriculan 2 veces al año, por eso se elimina duplicados
termino_duracion_carrera = matriculados[matriculados["cursando"]==0][["cat_periodo", "codigo_unico", "mrun", "anio_ing_carr_ori"]].drop_duplicates()

In [23]:
# Se corrobora que solo hay 1 año de ingreso a una carrera por alumno por carrera
ai = termino_duracion_carrera.groupby(["codigo_unico","mrun"]).agg({"anio_ing_carr_ori": pd.Series.nunique}).reset_index()
ai["anio_ing_carr_ori"].describe()

count    1.385502e+06
mean     1.015913e+00
std      1.267622e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      4.000000e+00
Name: anio_ing_carr_ori, dtype: float64

In [24]:
ai.shape, ai[ai["anio_ing_carr_ori"]>1].shape, 31971/1489620 * 100

((1385502, 3), (21771, 3), 2.1462520642848513)

In [59]:
ai_sort = ai.drop(columns=["codigo_unico"]).drop_duplicates().sort_values(by=["anio_ing_carr_ori", "mrun"], ascending=False)
ai_sort.index=ai_sort["mrun"]
ai_sort[ai_sort["anio_ing_carr_ori"] > 1].drop(columns=["mrun"]).rename(columns={"anio_ing_carr_ori": "cantidad_anio_ing_ori_diferentes"})

,cantidad_anio_ing_ori_diferentes
mrun,
24146362.0,4
20804709.0,4
19893993.0,4
15903180.0,4
10386143.0,4
...,...
1708.0,2
1645.0,2
952.0,2


In [60]:
t = matriculados[(matriculados["codigo_unico"]=="I113S10C430J2V1") & (matriculados["mrun"]==24146362.0)]
t.index = t["cat_periodo"]
t.drop(columns=["cat_periodo"])

,codigo_unico,mrun,anio_ing_carr_ori,anio_ing_carr_act,tipo_inst_1,dur_estudio_carr,cursando
cat_periodo,,,,,,,
2016,I113S10C430J2V1,24146362.0,2016,NaN,Institutos Profesionales,5,0
2017,I113S10C430J2V1,24146362.0,2014,2017.0,Institutos Profesionales,5,0
2018,I113S10C430J2V1,24146362.0,2015,2017.0,Institutos Profesionales,5,0
2019,I113S10C430J2V1,24146362.0,2017,2017.0,Institutos Profesionales,5,0


In [27]:
matriculados[(matriculados["codigo_unico"].isin(["I113S10C430J2V1", "I214S20C19J1V2"])) & (matriculados["mrun"].isin([24146362.0, 15903180.0]))].sort_values(by=["codigo_unico"])

,cat_periodo,codigo_unico,mrun,anio_ing_carr_ori,anio_ing_carr_act,tipo_inst_1,dur_estudio_carr,cursando
1033032,2016,I113S10C430J2V1,24146362.0,2016,NaN,Institutos Profesionales,5,0
1546685,2017,I113S10C430J2V1,24146362.0,2014,2017.0,Institutos Profesionales,5,0
2056751,2018,I113S10C430J2V1,24146362.0,2015,2017.0,Institutos Profesionales,5,0
2573541,2019,I113S10C430J2V1,24146362.0,2017,2017.0,Institutos Profesionales,5,0
1377554,2017,I214S20C19J1V2,15903180.0,2017,2017.0,Centros de Formación Técnica,3,0
2404324,2019,I214S20C19J1V2,15903180.0,2018,2018.0,Centros de Formación Técnica,3,0
2905998,2020,I214S20C19J1V2,15903180.0,2019,2019.0,Centros de Formación Técnica,3,0
3410532,2021,I214S20C19J1V2,15903180.0,2016,2016.0,Centros de Formación Técnica,3,0


In [28]:
matriculados[matriculados["mrun"].isin([24146362.0])]

,cat_periodo,codigo_unico,mrun,anio_ing_carr_ori,anio_ing_carr_act,tipo_inst_1,dur_estudio_carr,cursando
507008,2015,I113S10C435J2V1,24146362.0,2014,NaN,Institutos Profesionales,5,0
1033032,2016,I113S10C430J2V1,24146362.0,2016,NaN,Institutos Profesionales,5,0
1546685,2017,I113S10C430J2V1,24146362.0,2014,2017.0,Institutos Profesionales,5,0
2056751,2018,I113S10C430J2V1,24146362.0,2015,2017.0,Institutos Profesionales,5,0
2573541,2019,I113S10C430J2V1,24146362.0,2017,2017.0,Institutos Profesionales,5,0


In [29]:
termino_anio_ori_unico = termino_duracion_carrera[
    (termino_duracion_carrera["codigo_unico"].isin(ai[ai["anio_ing_carr_ori"]>1]["codigo_unico"]))
    & (termino_duracion_carrera["mrun"].isin(ai[ai["anio_ing_carr_ori"]>1]["mrun"]))
]

In [30]:
t = termino_duracion_carrera.groupby(["codigo_unico", "mrun"]).size().reset_index(name="cantidad_de_anios_cursados_por_carrera")
t[(t["codigo_unico"]=="I113S10C430J2V1") & (t["mrun"]==24146362.0)]

,codigo_unico,mrun,cantidad_de_anios_cursados_por_carrera
317556,I113S10C430J2V1,24146362.0,4


In [31]:
t2 = termino_anio_ori_unico.groupby(["codigo_unico", "mrun"]).size().reset_index(name="cantidad_de_anios_cursados_por_carrera")
t2[(t2["codigo_unico"]=="I113S10C430J2V1") & (t["mrun"]==24146362.0)]

/tmp/ipykernel_3966/1634338879.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  t2[(t2["codigo_unico"]=="I113S10C430J2V1") & (t["mrun"]==24146362.0)]


,codigo_unico,mrun,cantidad_de_anios_cursados_por_carrera


In [32]:
anios_matriculado = pd.merge(
    pd.merge(
        pd.merge(
            termino_duracion_carrera.groupby( 
                ["codigo_unico", "mrun"]
            ).size().reset_index(name="cantidad_de_anios_cursados_por_carrera"),
            matriculados[["codigo_unico", "dur_estudio_carr"]].drop_duplicates(),
            how="left"
        ),
        termino_duracion_carrera.groupby(["codigo_unico","mrun"])["cat_periodo"].max().reset_index().rename(columns={"cat_periodo": "ultimo_anio_matricula"}),
        how="left"
    ),
    termino_duracion_carrera.groupby(["codigo_unico","mrun"])["anio_ing_carr_ori"].max().reset_index().rename(columns={"anio_ing_carr_ori": "primer_anio_matricula"}),
    how="left"
)

anios_matriculado["duracion_carrera_anios"] = anios_matriculado["dur_estudio_carr"] / 2

anios_matriculado.drop(columns=["dur_estudio_carr"]).sort_values(by=["cantidad_de_anios_cursados_por_carrera"], ascending=False)

,codigo_unico,mrun,cantidad_de_anios_cursados_por_carrera,ultimo_anio_matricula,primer_anio_matricula,duracion_carrera_anios
644343,I143S15C17J1V1,11040267.0,8,2022,2015,3.5
442137,I123S1C25J1V1,23044049.0,8,2022,2014,5.0
271242,I111S4C6J1V1,991300.0,8,2022,2015,4.0
461242,I123S2C30J1V1,6597333.0,8,2022,2015,5.0
461162,I123S2C30J1V1,4090255.0,8,2022,2015,5.0
...,...,...,...,...,...,...
187113,I111S20C471J1V1,22504462.0,1,2019,2018,2.5
187112,I111S20C471J1V1,22436136.0,1,2017,2017,2.5
187111,I111S20C471J1V1,22203391.0,1,2019,2019,2.5
187109,I111S20C471J1V1,21323635.0,1,2017,2016,2.5


In [33]:
# Se debe marcar coo desertor al alumno en el último año cursado

In [34]:
#### DEBUGS

In [34]:
1589215 

1589215

In [35]:
matriculados["mrun"].value_counts().head()

231557.0      16
16639641.0    15
18235623.0    15
7085589.0     15
15827984.0    15
Name: mrun, dtype: int64

In [36]:
matriculados[matriculados["mrun"]==7085589.0].sort_values(by=["mrun", "cat_periodo"]).head(10)

,cat_periodo,codigo_unico,mrun,anio_ing_carr_ori,anio_ing_carr_act,tipo_inst_1,dur_estudio_carr,cursando
149129,2015,I492S0C4J2V1,7085589.0,2014,NaN,Centros de Formación Técnica,5,0
149130,2015,I117S1C35J4V1,7085589.0,2015,NaN,Institutos Profesionales,5,0
149131,2015,I176S9C778J3V1,7085589.0,2014,NaN,Institutos Profesionales,5,0
149132,2015,I116S39C51J2V1,7085589.0,2015,NaN,Institutos Profesionales,8,0
674466,2016,I162S2C35J2V1,7085589.0,2016,NaN,Institutos Profesionales,6,0
1197016,2017,I103S1C94J3V1,7085589.0,2017,NaN,Institutos Profesionales,5,0
1197017,2017,I120S6C12J2V1,7085589.0,2017,2017.0,Institutos Profesionales,8,0
1709754,2018,I143S28C9J4V1,7085589.0,2018,2018.0,Institutos Profesionales,5,0
2223032,2019,I171S1C39J2V4,7085589.0,2019,2019.0,Institutos Profesionales,8,1
2733946,2020,I214S1C49J4V2,7085589.0,2019,2019.0,Centros de Formación Técnica,3,0
